# Model to Predict Elo Customer Loyalty

_Note! If you want to commit any changes to this document, please strip all output (Cell > Current Outputs > Clear, or set up [nbstripout](https://github.com/kynan/nbstripout) as a git filter) from this notebook before doing so. Thanks!_


## Import Libraries

Next we import the Python libraries we'll need. If any of these are missing for you, you can install them with e.g. `pip3 install pandas` on the command line.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

## Load Data

Load the data into Pandas data frames and look at their structure.

First thing we'll do with the training data is split it into a train and validation set. (The given test set is what we'll later make our predictions on and upload, but only after we are fully satisfied with our model.)

In [ ]:
hist_trans_df = pd.read_csv('data/unzipped/historical_transactions.csv',
                            parse_dates=['purchase_date'])
merchants_df = pd.read_csv('data/unzipped/merchants.csv',
                           index_col='merchant_id')
merch_trans_df = pd.read_csv('data/unzipped/new_merchant_transactions.csv',
                             parse_dates=['purchase_date'])
train_and_validation_df = pd.read_csv('data/unzipped/train.csv',
                                      index_col='card_id',
                                      parse_dates=['first_active_month'])
test_df = pd.read_csv('data/unzipped/test.csv',
                      index_col='card_id',
                      parse_dates=['first_active_month'])

In [ ]:
hist_trans_df.head()

In [ ]:
merchants_df.head()

In [ ]:
merch_trans_df.head()

In [ ]:
train_and_validation_df.head()

## Create Features

Next we want to combine and shape all of our raw data to create useful features in the train data set.

For now, I'll do this step-by-step here to make it a bit easier to understand (for myself), but later we'll probably want to do this in some python classes, or this file will be huge. // Erich

In [ ]:
def add_aggregated_numerical_fields(df, hist_trans_df, column_names, aggregator):
    merged = df.merge(hist_trans_df, how='left', on=['card_id'])
    aggregated = merged.groupby('card_id')[column_names].agg([aggregator])
    for col in column_names:
        df[f'{col}_{aggregator.__name__}'] = aggregated[col]

In [ ]:
add_aggregated_numerical_fields(train_and_validation_df,
                                hist_trans_df,
                                column_names=['purchase_amount', 'installments', 'month_lag'],
                                aggregator=np.mean)

In [ ]:
add_aggregated_numerical_fields(train_and_validation_df,
                                hist_trans_df,
                                column_names=['purchase_amount', 'installments'],
                                aggregator=np.sum)

For the categorical fields, we can't aggregate by taking the mean or sum values, so let's count the occurences of each possible categorical value instead. _(Iow, for a category that can be either YES or NO, we count the number of YESes and the number of NOs and use those values.)_

In [ ]:
def add_aggregated_categorical_fields(df, hist_trans_df, column_names):
    merged = df.merge(hist_trans_df, how='left', on=['card_id'])
    for col in column_names:
        values = hist_trans_df[col].unique()
        counts = merged.groupby(['card_id', col]).size().unstack(fill_value=0) # 0 is not always a good default.
        total = counts.sum(axis=1) # Get total occurences of all values for the column.
        for value in values:
            if not pd.isnull(value): # Ignore nan. We should maybe handle this in a better way.
                df[f'{col}_{value}_count'] = counts[value]
                df[f'{col}_{value}_ratio'] = counts[value] / total

In [ ]:
add_aggregated_categorical_fields(train_and_validation_df,
                                  hist_trans_df,
                                  column_names=['authorized_flag', 'category_1', 'category_2', 'category_3'])

In [ ]:
train_and_validation_df.head()

### Repeat for Test Set

In [ ]:
add_aggregated_numerical_fields(test_df,
                                hist_trans_df,
                                column_names=['purchase_amount', 'installments', 'month_lag'],
                                aggregator=np.mean)
add_aggregated_numerical_fields(test_df,
                                hist_trans_df,
                                column_names=['purchase_amount', 'installments'],
                                aggregator=np.sum)
add_aggregated_categorical_fields(test_df,
                                  hist_trans_df,
                                  column_names=['authorized_flag', 'category_1', 'category_2', 'category_3'])

## Split Into Train and Validation Sets

Split our data into a train test (80%) and a validation set (20%).

In [ ]:
from sklearn.model_selection import train_test_split
train_df, validate_df = train_test_split(train_and_validation_df, test_size=0.2, random_state=238923)

In [ ]:
train_df.shape

In [ ]:
validate_df.shape

In [ ]:
train_df.head()

## Explore Data

### Correlations

In [ ]:
sns.heatmap(train_df.corr(), vmin=-1, vmax=1, cmap='PiYG', xticklabels=True, yticklabels=True)

### Distributions

In [ ]:
sns.distplot(train_df.target)

In [ ]:
sns.countplot(x='feature_1', palette='Set2', data=train_df)

In [ ]:
sns.countplot(x='feature_2', palette='Set2', data=train_df)

In [ ]:
sns.countplot(x='feature_3', palette='Set2', data=train_df)

In [ ]:
sns.distplot(train_df.purchase_amount_mean)

In [ ]:
sns.distplot(train_df.installments_mean)

In [ ]:
sns.distplot(train_df.month_lag_mean)

In [ ]:
sns.distplot(train_df.authorized_n_count)

## Set Up Model

We'll use the fastai tabular regressor, for which we'll need some additional imports.

In [ ]:
from fastai import *
from fastai.tabular import *
from fastai.metrics import *

### Create Data Bunch

A fastai DataBunch more or less contains the data that we'll feed to our model.

First, as the data bunch takes one data frame containing both the test and validation samples, we need to get the indices for our validation samples.

Then we tell the model which of the columns are categorical features, which are continuous features, and also which of the columns contains the target (the value we want to predict).

In [ ]:
valid_idx = range(len(train_and_validation_df) - len(validate_df), len(train_and_validation_df)); valid_idx

Puh, we have quite a lot of continuous features now. Rather than writing out all of them, let's just take all the columns and then subtract those that are _not_ continuous.

In [ ]:
category_names = ['feature_1', 'feature_2', 'feature_3']
dep_var = 'target'
continuous_names = [col for col in train_df.columns if col not in (
    ['first_active_month'] + category_names + [dep_var])]

Since we picked our validation samples randomly from the initial data set, and since fastai requires us to give the indices of the validation samples in a data frame containing both the training and validation samples, we just concatenate them together with training samples first and the validation samples at the end.

In [ ]:
df = pd.concat([train_df, validate_df]).reset_index()[category_names + continuous_names + [dep_var]]

In [ ]:
data = (TabularList.from_df(df,
                            path='data/unzipped',
                            cat_names=category_names,
                            cont_names=continuous_names,
                            procs=[FillMissing, Categorify, Normalize])
                .split_by_idx(valid_idx)
                .label_from_df(cols=dep_var, label_cls=FloatList)
                .databunch())

Let's have a look at a random batch of data to see how it looks after the processing done by the fastai library.

In [ ]:
data.show_batch()

### Create Learner

This is what we actually use to train the model and make predictions.

First we decide how large we want to make the embeddings of our categorical features (the number of category options divided by 2 is a good heuristic, apparently).

Then we tell the model the range within which we expect all predictions to fall (internally the model uses a sigmoid function, so in order for us, in practice, to actually get predictions near the expected maximum value, we set the upper bound to be a little higher than the expected maximum).

The competition uses root mean squared error to evaluate the entries, so we'll use that, too.

In [ ]:
category_szs = {'feature_1': 5,
                'feature_2': 3,
                'feature_3': 2}
emb_szs = {k: (v + 1) // 2 for k, v in category_szs.items()}; emb_szs

In [ ]:
max_y = np.max(train_df['target']) * 1.2
y_range = torch.tensor([-max_y, max_y], device=defaults.device); y_range

In [ ]:
np.max(train_df['target'])

In [ ]:
learn = tabular_learner(data,
                        layers=[1000, 500],
                        emb_szs=emb_szs,
                        ps=[1e-2, 1e-1],
                        emb_drop=0.05,
                        y_range=y_range,
                        metrics=rmse)

In [ ]:
learn.model

### Figure Out Learning Rate

To figure out which learning rate to use, we use fastai's learning rate finder.

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

### Train Model

Finally we train the model, with weight decay to encourage the model to use fewer features, and then show some results.

In [ ]:
learn.fit_one_cycle(1, 1e-5, wd=0.2)

In [ ]:
learn.recorder.plot_losses()

## Make Predictions

Now that we have trained our model, lets make some predictions to see whether or not our metrics lie to us.

In [ ]:
predictions, targets = [x.numpy().flatten() for x in learn.get_preds(DatasetType.Valid)]
prediction_df = pd.DataFrame({'prediction': predictions, 'target': targets})

In [ ]:
(np.amin(predictions), np.amax(predictions))

In [ ]:
prediction_df.head()

In [ ]:
prediction_df.tail()

### Calculate RMSE On Validation Set

Get the root mean squared error for the validation set only. This value we can compare against the public leaderboard on Kaggle, more or less.

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

In [ ]:
sqrt(mean_squared_error(prediction_df.target, prediction_df.prediction))

## Make Submission Predictions

Finally, we need to run our model against the test set that is used by the competition's organizers to evaluate the competitors. We save the result to a `submission.csv` file which we'll then upload to Kaggle.

_Note: we should only do this at the very end, when we are happy with our hyperparameters. Otherwise, if we change our model based on our results on the public leaderboard, we risk overfitting our model to the 30% of samples used for the public leaderboard, and will fail to generalize for the remaining 70% of samples._

In [ ]:
out_df = test_df.copy(); out_df.head()

In [ ]:
# Warning -- this takes quite a long time.
out_df['target'] = [learn.predict(row)[2].numpy().flatten()[0] for _, row in out_df.iterrows()]

In [ ]:
out_df['target'].to_csv('submission.csv.zip', header=['target'], index_label='card_id', compression='zip')